# Implementation of Basic Homebrew CNN

In [42]:
# General imports 
import sys
import os 
sys.path.insert(1, os.path.join(os.pardir, 'src'))
from itertools import product

# Data imports
import torch
import plotly.express as px
import numpy as np
import mlflow
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# Homebrew imports 
import model
from utils import one_hot_encode_index
from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer, ConvolutionLayer, PoolingLayer, FlattenLayer
from loss import CategoricalCrossEntropyLoss

## TESTING 
import importlib
importlib.reload(model)
##

<module 'model' from '..\\src\\model.py'>

## Data loaders

In [43]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(32),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(33),
                                      transforms.CenterCrop(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                    ])

# setting up data loaders
data_dir = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_32')

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'validation'), transform=test_transforms)

### Train config

In [44]:
# Configs 
config = {
    'max_epochs': 10,
    'learning_rate': 0.003,
    'resolution': 32,
    'name': 'CNN_Basic_1CNN_module_homebrew'
}

## Model

In [45]:
mdl = model.Model(Adam(learning_rate=config['learning_rate']),
                      CategoricalCrossEntropyLoss())

# Config early stop 
mdl.add_early_stop(5)

mdl.set_save_config(model_name=config['name'], save_path=os.path.join(os.pardir, 'models'))

# Defining architecture 

mdl.set_sequence([
                    ConvolutionLayer(3, 32, 3),
                    ReLU(),
                    PoolingLayer(32, 2),
                    ConvolutionLayer(31, 16, 3),
                    ReLU(),
                    PoolingLayer(16, 2),
                    FlattenLayer(),
                    LinearLayer(784, 1568),
                    ReLU(),
                    LinearLayer(1568, 128),
                    ReLU(),
                    LinearLayer(128, 39),
                    Softmax()
                ])
print(mdl)

Model Architecture: 
	 (0): ConvolutionLayer (Trainable: False)
	 (1): ReLU (Trainable: False)
	 (2): PoolingLayer (Trainable: False)
	 (3): ConvolutionLayer (Trainable: False)
	 (4): ReLU (Trainable: False)
	 (5): PoolingLayer (Trainable: False)
	 (6): FlattenLayer (Trainable: False)
	 (7): LinearLayer (Trainable: True)
	 (8): ReLU (Trainable: False)
	 (9): LinearLayer (Trainable: True)
	 (10): ReLU (Trainable: False)
	 (11): LinearLayer (Trainable: True)
	 (12): Softmax (Trainable: False)



## Training 

In [46]:
mlflow.set_experiment("Plant Leaf Disease")

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)


with mlflow.start_run():
    mlflow.log_param('framework', 'homebrew')
    mlflow.log_param('data_split', '90/10')
    mlflow.log_param('type', 'Basic_CNN')
    mlflow.log_params(config)
    mdl.train_with_loader(train_loader, epochs=config['max_epochs'], validation_loader=validation_loader, cls_count=39, log_freq=5)

=== Epoch: 1 ===
Step: 0/865, accuracy0.016, loss3.729, learning rate 0.0030000 


In [ ]:
mdl.save(os.path.join(os.pardir, 'models', 'basic_bkup'))

(64, 784)